In [3]:
pip install dlt[duckdb]

  Obtaining dependency information for dlt[duckdb] from https://files.pythonhosted.org/packages/5b/0a/3bcfec9a96d8f77b0cd8e3ce34ab452a92bcddfb5398b008e2ea2ae2a5a0/dlt-0.4.4-py3-none-any.whl.metadata
  Obtaining dependency information for duckdb<0.10.0,>=0.6.1 from https://files.pythonhosted.org/packages/a6/7a/e15904563fa63d7d3cc542a697377486ddea8ba4914810391812ffefc9b1/duckdb-0.9.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for gitpython>=3.1.29 from https://files.pythonhosted.org/packages/67/c7/995360c87dd74e27539ccbfecddfb58e08f140d849fcd7f35d2ed1a5f80f/GitPython-3.1.42-py3-none-any.whl.metadata
  Obtaining dependency information for giturlparse>=0.10.0 from https://files.pythonhosted.org/packages/dd/94/c6ff3388b8e3225a014e55aed957188639aa0966443e0408d38f0c9614a7/giturlparse-0.12.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for hexbytes>=0.2.2 from https://files.pythonhosted.org/packages/73/99/f6beab7c3fe2ad62f6763a5e92af476225d74408b2

In [4]:
import dlt

# Use a generator

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

In [13]:
# Example usage:
limit = 5
generator = square_root_generator(limit)
r=0

for sqrt_value in generator:
    r=r+sqrt_value

print(r)

8.382332347441762


In [14]:
# Example usage:
limit = 13
generator = square_root_generator(limit)
r=0

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# Append a generator to a table with existing data

In [64]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}


In [63]:
r=0
for person in people_1():
    r=r+person['Age']
print(r)

140


In [18]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [115]:
# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='peopleset')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1,
                    table_name="peopleset",
                    write_disposition="replace",
                    primary_key='ID')

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset peopleset
The duckdb destination used duckdb:///C:\Users\scl\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708155976.3690548 is LOADED and contains no failed jobs


In [116]:
# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='peopleset')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_2,
                    table_name="peopleset",
                    write_disposition="append",
                    primary_key='ID')

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset peopleset
The duckdb destination used duckdb:///C:\Users\scl\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708155985.7650282 is LOADED and contains no failed jobs


In [117]:
# show outcome

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n People table below: Note the times are properly typed")
people = conn.sql("SELECT * FROM peopleset").df()
display(people)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ peopleset           │
└─────────────────────┘




 People table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708155976.3690548,8dfZK6jBldHQcw,None
1,2,Person_2,27,City_A,1708155976.3690548,NMzvDDkKbS8ttg,None
2,3,Person_3,28,City_A,1708155976.3690548,72QTgAqWh6l0oA,None
3,4,Person_4,29,City_A,1708155976.3690548,OMzrqC/T5FpFqA,None
4,5,Person_5,30,City_A,1708155976.3690548,dNnUqt6Y/0Jw/w,None
5,3,Person_3,33,City_B,1708155985.7650282,WuRGyfN4wA3rJA,Job_3
6,4,Person_4,34,City_B,1708155985.7650282,Klh8P1S5palEVw,Job_4
7,5,Person_5,35,City_B,1708155985.7650282,XuVUukMV3s79Tg,Job_5
8,6,Person_6,36,City_B,1708155985.7650282,cSgrX9/wYsIcAA,Job_6
9,7,Person_7,37,City_B,1708155985.7650282,7Bv2dOhM/evTcg,Job_7


In [118]:
age = conn.sql("select sum(age) from peopleset").df()
display(age)

,sum(age)
0,353.0


In [110]:
# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='merge_people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1,
                    table_name="merge_people",
                    write_disposition="replace",
                    primary_key='ID')

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset merge_people
The duckdb destination used duckdb:///C:\Users\scl\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708155801.526567 is LOADED and contains no failed jobs


In [112]:
# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='merge_people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_2,
                    table_name="merge_people",
                    write_disposition="merge",
                    primary_key='ID')

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset merge_people
The duckdb destination used duckdb:///C:\Users\scl\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708155875.2275379 is LOADED and contains no failed jobs


In [113]:

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n People table below: Note the times are properly typed")
people = conn.sql("SELECT * FROM merge_people").df()
display(people)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ merge_people        │
└─────────────────────┘




 People table below: Note the times are properly typed


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708155801.526567,DfzfSekPk1OqCw,None
1,2,Person_2,27,City_A,1708155801.526567,zTF8eMIZeB632g,None
2,8,Person_8,38,City_B,1708155875.2275379,pddXHugbidBPYg,Job_8
3,4,Person_4,34,City_B,1708155875.2275379,4oA6ME7zsdftLw,Job_4
4,5,Person_5,35,City_B,1708155875.2275379,uWTFKnoX8s/Gfw,Job_5
5,3,Person_3,33,City_B,1708155875.2275379,H7oSf34l0PG+Bg,Job_3
6,6,Person_6,36,City_B,1708155875.2275379,XGn5dbT+08YD+Q,Job_6
7,7,Person_7,37,City_B,1708155875.2275379,h/32mjx5OfGK2A,Job_7


In [114]:
merge_age = conn.sql("select sum(age) from merge_people").df()
display(merge_age)

,sum(age)
0,266.0
